In [1]:
import sys
sys.executable

'/opt/homebrew/Caskroom/miniconda/base/envs/bioenv/bin/python'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import pandas as pd
import spacy
import datasets
from datasets import Dataset
import tqdm
import evaluate
from collections import Counter, defaultdict
import pyabf
import ast
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
# read the contents of physionet
import wfdb
import scipy.io

# Reading the .hea file
record = wfdb.rdheader('/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/cpsc_2018/g1/A0001')

# Display the header information
print(record)

# Reading the .mat file
mat_data = scipy.io.loadmat('/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/cpsc_2018/g1/A0001.mat')

# Display the .mat file content
print(mat_data)


{'val': array([[  28,   39,   45, ...,  258,  259,  259],
       [   7,   11,   15, ...,  248,  249,  250],
       [ -21,  -28,  -30, ...,  -10,  -10,   -9],
       ...,
       [-112, -110, -108, ...,  194,  194,  195],
       [-596, -590, -582, ...,  307,  307,  307],
       [ -16,   -7,    2, ...,  213,  214,  214]], dtype=int16)}


In [ ]:
'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/cpsc_2018_extra/g1/Q0001.hea'
'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/cpsc_2018_extra/g1/Q0001.mat'

'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/georgia/g1/E00001.hea'
'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/georgia/g1/E00001.mat'

'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/ptb/g1/S0001.hea'
'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/ptb/g1/S0001.mat'

'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/ptb-xl/g1/HR00001.hea'
'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/ptb-xl/g1/HR00001.mat'

'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/st_petersburg_incart/g1/I0001.hea'
'/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/st_petersburg_incart/g1/I0001.mat'





In [3]:
print(dir(record))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adjust_datetime', '_arrange_fields', '_auto_signal_file_names', 'adc', 'adc_gain', 'adc_res', 'adc_zero', 'base_counter', 'base_date', 'base_datetime', 'base_time', 'baseline', 'block_size', 'byte_offset', 'calc_adc_params', 'calc_checksum', 'check_field', 'check_field_cohesion', 'check_read_inputs', 'check_sig_cohesion', 'checksum', 'comments', 'convert_dtype', 'counter_freq', 'd_signal', 'dac', 'e_d_signal', 'e_p_signal', 'file_name', 'fmt', 'fs', 'get_absolute_time', 'get_elapsed_time', 'get_frame_number', 'get_write_fields', 'get_write_subset', 'init_value', 'n_sig', 'p_signal', 'record_name', 'samps_per_frame', 'set_d_features', 'set_defaul

In [5]:
import wfdb

# Read the .hea file
record = wfdb.rdheader('/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/cpsc_2018/g1/A0001')

# List of attributes and methods to print
attributes = dir(record)
# Print the attributes and their values
for attr in attributes:
    try:
        value = getattr(record, attr)
        print(f"{attr}: {value}\n")
    except Exception as e:
        print(f"{attr}: Could not retrieve value ({e})\n")


print(record[comments


__class__: <class 'wfdb.io.record.Record'>

__delattr__: <method-wrapper '__delattr__' of Record object at 0x104447cd0>

__dict__: {'record_name': 'A0001', 'n_sig': 12, 'fs': 500, 'counter_freq': None, 'base_counter': None, 'sig_len': 7500, 'base_time': None, 'base_date': None, 'comments': ['Age: 74', 'Sex: Male', 'Dx: 59118001', 'Rx: Unknown', 'Hx: Unknown', 'Sx: Unknown'], 'sig_name': ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'], 'p_signal': None, 'd_signal': None, 'e_p_signal': None, 'e_d_signal': None, 'file_name': ['A0001.mat', 'A0001.mat', 'A0001.mat', 'A0001.mat', 'A0001.mat', 'A0001.mat', 'A0001.mat', 'A0001.mat', 'A0001.mat', 'A0001.mat', 'A0001.mat', 'A0001.mat'], 'fmt': ['16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16'], 'samps_per_frame': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'skew': [None, None, None, None, None, None, None, None, None, None, None, None], 'byte_offset': [24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24], '

In [8]:
import wfdb

# Read the .hea file
record = wfdb.rdheader('/Users/b/bio/cardiac/model2/physionet.org/files/challenge-2020/1.0.2/training/cpsc_2018/g1/A0001')

# Access the dictionary containing the record's attributes
record_dict = record.__dict__

# Extract the comments
comments = record_dict['comments']

# Print all comments
print("Comments:")
for comment in comments:
    print(comment)

# Extract the 'Dx' (Diagnosis) from the comments
dx_comment = next((comment for comment in comments if comment.startswith('Dx:')), None)

# Print the 'Dx' value
if dx_comment:
    print("\nDiagnosis (Dx):", dx_comment.split('Dx: ')[1])
else:
    print("\nDiagnosis (Dx) not found.")

print(dx_comment.split('Dx: ')[1])


Comments:
Age: 74
Sex: Male
Dx: 59118001
Rx: Unknown
Hx: Unknown
Sx: Unknown

Diagnosis (Dx): 59118001
59118001


In [10]:
# create the dataset, 
record = wfdb.rdheader('physionet.org/files/challenge-2020/1.0.2/training/cpsc_2018/g1/A0001')

In [ ]:
file = 'physionet.org/files/challenge-2020/1.0.2/SHA256SUMS.txt'
# read file and create the dataset

contains contents

In [13]:
len(mat_data.tolist())

AttributeError: 'dict' object has no attribute 'tolist'

In [22]:
len(list(mat_data.values())[0])

12

In [23]:
import os
import wfdb
import scipy.io

In [ ]:
# Path to the SHA256SUMS.txt file
sha256_file_path = 'physionet.org/files/challenge-2020/1.0.2/SHA256SUMS.txt'

data = []
empty_list = 0
empty_diagnosis = 0

# Read the SHA256SUMS.txt file
with open(sha256_file_path, 'r') as file:
    lines = file.readlines()

for line in lines:
    parts = line.split()
    file_path = parts[1] if len(parts) > 1 else None

    if file_path and file_path.startswith('training') and file_path.endswith('.hea'):
        file_path = 'physionet.org/files/challenge-2020/1.0.2/' + file_path
        file_path = file_path[:-4]
        hea_record = wfdb.rdheader(file_path)

        file_path = file_path + ".mat"
        mat_file_path = file_path.replace('.hea', '.mat')

        mat_data = scipy.io.loadmat(mat_file_path)

         # Flatten the data from the .mat file (assuming the data is in 'val')
        input_data = mat_data['val'].flatten().tolist() if 'val' in mat_data else []
        empty_list += 0 if len(input_data)>0 else 1
        
        # Extract the diagnosis from the .hea file (assuming 'Dx' is stored in comments)
        diagnosis = None
        for comment in hea_record.comments:
            if comment.startswith('Dx:'):
                diagnosis = comment.split('Dx: ')[1]
                break
        empty_diagnosis += 1 if diagnosis else 0
        
        # Create a dictionary with "input" and "output" keys
        record_dict = {
            "input": input_data,
            "output": diagnosis
        }
        
        # Add the dictionary to the list
        data.append(record_dict)
